In [1]:
import pandas as pd
import numpy as np
import json
import ast
import itertools 
from simulation_utils import *

## Experiment 1: Human Navigation Behavior in Grid World of Size 6*6

In [2]:
# Load navigation data and map layouts
grid_size = 6
experiment1_data = pd.read_csv('../experiment1/experiment1-preprocessed.csv', usecols=['idx', 'move'])
with open('../experiment1/map_layout_experiment1.json', 'r') as json_file:
    experiment1_layouts = json.load(json_file)

In [3]:
# Initialize environment and process data
successful_games = []
env = navigation_env(grid_size)
for idx, row in experiment1_data.iterrows():
    game_idx, moves = row['idx'], json.loads(row['move'])
    env.load_layout(experiment1_layouts[game_idx])
    successful_games.append(env.navigate(moves))

# Display results
print('Total games: ', len(successful_games))
print('Successful games: ', np.sum(successful_games))

Total games:  3000
Successful games:  2980


## Experiment 2: Human Goal Inference in Grid World of Size 6*6

- treatment: 0-shortest distance, 1-behavioral cloning, 2-human data (doesn't contain true goals)

In [4]:
# Load inference data and map layouts
grid_size = 6
experiment2_data = pd.read_csv('../experiment2/experiment2-preprocessed.csv', usecols=['idx', 'answer', 'treatment'])
with open('../experiment2/map_layout_experiment2.json', 'r') as json_file:
    experiment2_layouts = json.load(json_file)

In [5]:
# Display the first few rows of the data
experiment2_data.head()
# Columns:
# - idx: The game index of layouts
# - answer: Human inference of goals
# - treatment: 0 for shortest distance (generated by AI), 1 for behavioral cloning (generated by AI), 2 for human behavior data

,idx,answer,treatment
0,20,1,2
1,22,0,2
2,4,1,2
3,6,1,2
4,1,0,2


In [6]:
# Display a sample layout dictionary
sample_layout = experiment2_layouts[0][0] ## the first layout for treatment 0
layout_dict = {
    'block': sample_layout[0],        # Grid configuration with blocks
    'start_pos': sample_layout[1],    # Start position
    'goal_pos': sample_layout[2],     # Two goal positions
    'trajectory': sample_layout[3],   # Path taken by the player as observed by human participants
    'designed_goal': sample_layout[4] if len(sample_layout) == 5 else -1 # Actual goal of the trajectory if available
}
print(layout_dict)

{'block': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0]], 'start_pos': [0, 0], 'goal_pos': [[2, 4], [5, 4]], 'trajectory': [[1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [5, 1], [4, 1]], 'designed_goal': 1}


## Experiment 3:  Human Navigation Behavior in Grid World of Size 8*8

In [7]:
# Load inference data and map layouts
grid_size = 8
experiment3_data = pd.read_csv('../experiment3/experiment3-preprocessed.csv')
with open('../experiment3/map_layout_experiment3.json', 'r') as json_file:
    experiment3_layouts = json.load(json_file)

In [8]:
# Display the first few rows of the data
experiment3_data.head()
# Columns:
# - game_idx: Index of the game layout
# - ai_treatment: Treatment type of AI models (0: level-0 model, 1: self-play model, 2: optimal model)
# - hint_treatment: Type of hint provided (0: human's goal, 1: AI's goal)
# - is_tutorial_game: Indicates if the game is a tutorial (True) or a formal game (False)
# - reward: Rewards given in the game (-10: collision, +4: inconsistent goals reached, +10: consistent goals reached, 0: otherwise)
# - hit_goals: Actual goals reached by the two players (-1 indicates no goal reached)
# - designed_goals: Target goals for the two players
# - player0_move: Actions taken by player 0 (real human)
# - player1_move: Actions taken by player 1 (AI model)

,Unnamed: 0,game_idx,ai_treatment,hint_treatment,is_tutorial_game,reward,hit_goals,designed_goals,player0_move,player1_move
0,0,1001,1,1,True,10,"(1, 3)","(1, 3)","[2, 2, 2, 2]","[0, 3, 3, 3]"
1,1,1002,1,1,True,-10,"(-1, -1)","(0, 2)","[0, 0]","[1, 2]"
2,2,1003,1,1,True,-10,"(-1, -1)","(0, 2)","[0, 4, 4]","[1, 1, 3]"
3,3,586,1,1,False,10,"(0, 2)","(0, 2)","[3, 2, 3, 3, 1, 0, 0, 3]","[0, 0, 3, 3, 3, 3, 3, 1]"
4,4,210,1,1,False,10,"(1, 3)","(1, 3)","[0, 3, 3, 3, 2, 2, 2, 2, 0, 0, 2, 2]","[1, 1, 1, 3, 4, 4, 4, 4, 4, 4, 4, 4]"


In [9]:
# Initialize environment and process data

reward = np.zeros(len(experiment3_data))

for i in range(len(experiment3_data)):
    A = experiment3_data.iloc[i]
    idx = A['game_idx']
    seq = experiment3_layouts[idx]
    block_pos, start_pos, goal_pos, _ = seq
    start_pos = [tuple(pos) for pos in start_pos]
    goal_pos = [tuple(pos) for pos in goal_pos]
    env = navigation_share_env(grid_size, start_pos, goal_pos, block_pos)
    ans = env.navigate([json.loads(A['player0_move']), json.loads(A['player1_move'])])
    reward[i] = ans[0]

print(np.unique(reward, return_counts=True))

(array([-10.,   0.,   4.,  10.]), array([2405,  230, 3199, 7366]))


## Experiment 4: Human Goal Inference in Grid World of Size 8*8

In [10]:
# Load inference data and map layouts
grid_size = 8
experiment4_data = pd.read_csv('../experiment4/experiment4-preprocessed.csv')
with open('../experiment4/map_layout_experiment4.json', 'r') as json_file:
    experiment4_layouts = json.load(json_file)

In [11]:
# Display the first few rows of the data
experiment4_data.head()
# Columns:
# - human_belief: Goals selected by humans (0 or 1)
# - treatment: Type of trajectory treatment (0: shortest distance model, 1: behavioral cloning)
# - game_idx: Index of the game layout
# - designed_goals: Target goals for the player 0
# - player0_move: Actions taken by player 0 (humans are asked to infer goals of palyer 0)
# - player1_move: Actions taken by player 1

,Unnamed: 0,human_belief,treatment,game_idx,designed_goal,player0_move,player1_move
0,0,0,0,155,0,"[2, 2, 0, 0, 2, 2, 2]","[2, 2, 0, 0, 2, 2, 2]"
1,1,0,0,129,0,"[2, 0, 0, 0, 0]","[2, 0, 0, 0, 0]"
2,2,0,0,95,0,"[1, 1, 1, 1]","[1, 1, 1, 1]"
3,3,0,0,159,1,"[2, 0, 0, 0, 0, 0, 3]","[2, 0, 0, 0, 0, 0, 3]"
4,4,0,0,58,1,"[3, 0, 3, 3]","[3, 0, 3, 3]"


In [12]:
sample_layout = experiment4_layouts[0][0] ## the first layout for treatment 0
layout_dict = {
    'block': sample_layout[0],        # Grid configuration with blocks
    'start_pos': sample_layout[1],    # Start position
    'goal_pos': sample_layout[2],     # Goal positions, the fitst two goals are for player 0  
    'designed_goal': sample_layout[3] # Actual goal of the trajectory if available
}
print(layout_dict)

{'block': [[0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 1, 1, 0, 0, 1], [0, 1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 1, 0]], 'start_pos': [[3, 6], [1, 3]], 'goal_pos': [[6, 3], [4, 1], [6, 6], [2, 7]], 'designed_goal': [1, 3]}
